# Seminararbeit: Konzeption eines minimalen CI-Prozesses für dbt-Projekte

**Kurs:** WWI2023F  
**Dozent:** Titel, Vorname und Nachname  
**Autor:** Robert Knabe  
**Datum:** 2026-01-29

## Inhaltsverzeichnis

1. [Einleitung und Motivation](#1-einleitung-und-motivation)
   - 1.1 [Problemstellung](#11-problemstellung)
   - 1.2 [Zielsetzung der Arbeit](#12-zielsetzung-der-arbeit)
   - 1.3 [Aufbau der Arbeit](#13-aufbau-der-arbeit)
2. [Grundlagen und fachliche Einordnung](#2-grundlagen-und-fachliche-einordnung)
   - 2.1 [ETL und ELT im Data Engineering](#21-etl-und-elt-im-data-engineering)
   - 2.2 [Rolle von dbt in der Transformationsphase](#22-rolle-von-dbt-in-der-transformationsphase)
   - 2.3 [Continuous Integration im Datenkontext](#23-continuous-integration-im-datenkontext)
   - 2.4 [Observability und Datenqualität](#24-observability-und-datenqualität)
3. [Konzeption eines minimalen CI Prozesses](#3-konzeption-eines-minimalen-ci-prozesses)
   - 3.1 [Rahmenbedingungen und Annahmen](#31-rahmenbedingungen-und-annahmen)
   - 3.2 [Architekturübersicht](#32-architekturübersicht)
   - 3.3 [Designentscheidungen](#33-designentscheidungen)
4. [Technische Umsetzung](#4-technische-umsetzung)
   - 4.1 [Projektstruktur](#41-projektstruktur)
   - 4.2 [Datenmodellierung mit dbt](#42-datenmodellierung-mit-dbt)
   - 4.3 [Definition von Datenqualitätstests](#43-definition-von-datenqualitätstests)
   - 4.4 [CI Pipeline Ablauf](#44-ci-pipeline-ablauf)
   - 4.5 [Beispielhafter CI Lauf](#45-beispielhafter-ci-lauf)
5. [Kritische Würdigung](#5-kritische-würdigung)
   - 5.1 [Stärken des Ansatzes](#51-stärken-des-ansatzes)
   - 5.2 [Limitationen](#52-limitationen)
   - 5.3 [Abgrenzung zu produktiven Setups](#53-abgrenzung-zu-produktiven-setups)
6. [Fazit und Ausblick](#6-fazit-und-ausblick)

## 1. Einleitung und Motivation

### 1.1 Problemstellung

In modernen Data Engineering Architekturen werden Daten zunehmend automatisiert verarbeitet und transformiert. Die Qualität dieser Transformationen ist dabei entscheidend für die Verlässlichkeit nachgelagerter Analysen. Fehler in Transformationslogik oder inkonsistente Datenstrukturen können dazu führen, dass Analyseergebnisse verfälscht werden oder wichtige Geschäftsentscheidungen auf falschen Annahmen beruhen.

In vielen Projekten werden Datenmodelle manuell getestet oder Änderungen an Transformationsskripten ohne automatisierte Qualitätsprüfungen produktiv gesetzt. Dadurch entstehen Risiken hinsichtlich Datenqualität, Reproduzierbarkeit und Nachvollziehbarkeit. Besonders in ELT-Ansätzen, bei denen Transformationen direkt im Zielsystem stattfinden, ist eine strukturierte Qualitätssicherung erforderlich.

Continuous Integration ist in der klassischen Softwareentwicklung ein etabliertes Mittel zur automatisierten Prüfung von Codeänderungen. Im Data Engineering Kontext wird dieses Prinzip jedoch häufig nur eingeschränkt oder gar nicht angewendet. Gleichzeitig gewinnen Werkzeuge wie dbt an Bedeutung, die strukturierte SQL-basierte Transformationen und integrierte Datenqualitätstests ermöglichen.

Vor diesem Hintergrund stellt sich die Frage, wie ein minimaler, leichtgewichtiger CI-Prozess für dbt-Projekte gestaltet werden kann, um Datenqualität und Transparenz zu verbessern, ohne komplexe Infrastruktur oder hohe Betriebskosten zu verursachen.

### 1.2 Zielsetzung der Arbeit

Ziel dieser Arbeit ist es, ein einfaches und reproduzierbares Konzept für einen minimalen CI-Prozess für dbt-Projekte zu entwerfen und umzusetzen. Dabei soll gezeigt werden, wie durch automatisierte Ausführung von Transformationen und Datenqualitätstests eine grundlegende Form von Observability im Data Engineering erreicht werden kann.

Der Fokus liegt nicht auf einer produktionsreifen Architektur, sondern auf der konzeptionellen Einordnung, den getroffenen Designentscheidungen sowie deren Grenzen. Als technische Basis werden dbt, DuckDB und GitHub Actions verwendet, um einen vollständig lokalen und leichtgewichtigen ELT-Prozess mit automatisierter Qualitätssicherung abzubilden.

Die Arbeit verfolgt das Ziel, die Prinzipien von Continuous Integration auf den Data Engineering Kontext zu übertragen und kritisch zu bewerten, inwieweit ein solcher Ansatz zur Verbesserung von Datenqualität, Nachvollziehbarkeit und Reproduzierbarkeit beitragen kann.

### 1.3 Aufbau der Arbeit

Nach der Einleitung werden zunächst die relevanten fachlichen Grundlagen zu ETL- und ELT-Prozessen, dbt, Continuous Integration sowie Observability im Datenkontext vorgestellt. Anschließend wird das konzeptionelle Design des minimalen CI-Prozesses erläutert, inklusive der zugrunde liegenden Annahmen und Architekturentscheidungen.

Darauf aufbauend wird die technische Umsetzung des Projekts beschrieben, insbesondere die Struktur des dbt-Projekts, die Definition der Datenmodelle und Tests sowie der Ablauf der CI-Pipeline mit GitHub Actions. Abschließend erfolgt eine kritische Würdigung des Ansatzes, in der Stärken, Limitationen und Abgrenzungen zu produktiven Setups diskutiert werden. Ein Fazit mit Ausblick rundet die Arbeit ab.

## 2. Grundlagen und fachliche Einordnung

### 2.1 ETL und ELT im Data Engineering

Im Data Engineering haben sich zwei grundlegende Paradigmen zur Integration und Aufbereitung von Daten etabliert: ETL und ELT. ETL steht für Extract, Transform und Load und beschreibt einen klassischen Ansatz, bei dem Daten zunächst aus verschiedenen Quellen extrahiert, dann außerhalb des Zielsystems transformiert und anschließend in das Data Warehouse geladen werden. Dieser Ansatz war besonders in Zeiten verbreitet, in denen Speicherplatz teuer und Rechenkapazität in Zielsystemen begrenzt war.

Mit der Entwicklung moderner Cloud-Data-Warehouses wie Snowflake, BigQuery oder Redshift sowie lokaler Lösungen wie DuckDB hat sich das ELT-Paradigma zunehmend durchgesetzt. Hier werden Daten zunächst unverändert in das Zielsystem geladen und erst dort transformiert. Der Vorteil liegt in der Nutzung der skalierbaren Rechenkapazität moderner Datenbanksysteme sowie in der Nachvollziehbarkeit, da Rohdaten stets verfügbar bleiben.

ELT ermöglicht eine klarere Trennung der Verantwortlichkeiten: Datenintegration und Transformation werden als separate Prozesse behandelt. Dies erleichtert die Versionierung von Transformationslogik sowie die iterative Weiterentwicklung von Datenmodellen, da Änderungen direkt im Zielsystem nachvollzogen werden können. Zudem lassen sich Transformationen in SQL formulieren, wodurch die Einstiegshürde für Analystinnen und Analysten sinkt.

### 2.2 Rolle von dbt in der Transformationsphase

dbt (data build tool) ist ein Open-Source-Werkzeug, das speziell für die Transformationsphase in ELT-Prozessen entwickelt wurde. Es ermöglicht die Definition von Datenmodellen als SQL-Abfragen, die in einer festgelegten Reihenfolge ausgeführt werden. Diese Modelle können als Views oder als materialisierte Tabellen abgebildet werden.

Ein zentraler Vorteil von dbt liegt in der Modularisierung von Transformationslogik. Modelle können hierarchisch aufgebaut werden, wobei nachgelagerte Modelle auf vorgelagerte Modelle referenzieren. dbt übernimmt die Auflösung dieser Abhängigkeiten automatisch und erstellt einen gerichteten azyklischen Graphen (DAG), der die Ausführungsreihenfolge bestimmt.

Darüber hinaus bietet dbt integrierte Mechanismen zur Dokumentation und zum Testen von Datenmodellen. Tests können auf Spaltenebene oder auf Modellebene definiert werden und prüfen etwa auf fehlende Werte, Eindeutigkeit oder referenzielle Integrität. Diese Tests sind in der Projektstruktur verankert und werden bei jeder Ausführung mitgeprüft.

dbt fördert damit ein deklaratives Verständnis von Datenmodellierung: statt imperative Skripte zu schreiben, beschreiben Entwicklerinnen und Entwickler den gewünschten Zielzustand. Diese Denkweise ist gut mit modernen DevOps-Praktiken vereinbar und ermöglicht es, Qualitätssicherung als integralen Bestandteil des Transformationsprozesses zu verstehen.

### 2.3 Continuous Integration im Datenkontext

Continuous Integration (CI) bezeichnet in der Softwareentwicklung die Praxis, Codeänderungen regelmäßig und automatisiert zu integrieren sowie durch automatisierte Tests auf Korrektheit zu prüfen. Ziel ist es, Fehler frühzeitig zu erkennen und die Integrierbarkeit neuer Änderungen sicherzustellen.

Im Kontext von Data Engineering wird CI bislang weniger konsequent eingesetzt als in der klassischen Softwareentwicklung. Dabei lassen sich viele Prinzipien übertragen: Transformationsskripte sind Code und sollten entsprechend versioniert, getestet und geprüft werden. Eine CI-Pipeline für ein dbt-Projekt kann beispielsweise bei jeder Änderung automatisch alle Transformationen ausführen und definierte Datenqualitätstests durchlaufen.

Ein solcher Ansatz erhöht die Transparenz, da jede Änderung dokumentiert und geprüft wird. Gleichzeitig ermöglicht er es, frühzeitig Inkonsistenzen oder Fehler zu identifizieren, bevor sie in produktive Systeme gelangen. Insbesondere in kollaborativen Projekten trägt CI dazu bei, eine gemeinsame Qualitätsbasis zu schaffen.

Allerdings unterscheidet sich CI im Datenkontext in einigen Punkten von klassischer Softwareentwicklung. Daten ändern sich im Zeitverlauf, Tests sind oft kontextabhängig und die Ausführungsdauer kann deutlich höher sein. Dennoch bietet CI auch hier einen Mehrwert, indem es strukturierte Qualitätssicherung automatisiert und nachvollziehbar macht.

### 2.4 Observability und Datenqualität

Observability im Kontext von Datensystemen meint die Fähigkeit, den Zustand und die Qualität von Daten sowie die Funktionsweise von Transformationsprozessen nachvollziehen zu können. Dies umfasst Aspekte wie Monitoring, Logging, Tracing und Alerting. Ziel ist es, Probleme frühzeitig zu erkennen und fundierte Entscheidungen über Datenqualität und Systemverhalten treffen zu können.

Datenqualität lässt sich anhand verschiedener Dimensionen bewerten, etwa Vollständigkeit, Korrektheit, Konsistenz, Aktualität und Eindeutigkeit. In einem dbt-Projekt können diese Dimensionen durch gezielte Tests adressiert werden. Beispielsweise prüft ein Unique-Test die Eindeutigkeit von Primärschlüsseln, während ein Not-Null-Test die Vollständigkeit bestimmter Spalten sicherstellt.

Observability und Datenqualität sind eng miteinander verknüpft. Automatisierte Tests liefern kontinuierlich Informationen über den Zustand der Datenmodelle und erzeugen damit eine Form von passiver Observability. In Kombination mit einem CI-Prozess entsteht ein transparenter und nachvollziehbarer Workflow, der es ermöglicht, Qualitätsprobleme systematisch zu identifizieren und zu beheben.

Ein minimaler CI-Prozess mit dbt trägt somit nicht nur zur Qualitätssicherung bei, sondern erhöht auch die Sichtbarkeit von Datenprozessen. Dies ist besonders in frühen Entwicklungsphasen oder in kleineren Projekten von Vorteil, in denen aufwändige Monitoring-Lösungen unverhältnismäßig wären.

## 3. Konzeption eines minimalen CI Prozesses

### 3.1 Rahmenbedingungen und Annahmen

Die Konzeption des minimalen CI-Prozesses basiert auf der Annahme, dass ein lokales, leichtgewichtiges Setup ausreicht, um die grundlegenden Prinzipien von Continuous Integration im Datenkontext zu demonstrieren. Es wird bewusst auf cloud-basierte Infrastruktur verzichtet, um Komplexität und Kosten gering zu halten. Stattdessen wird eine vollständig dateibasierte Datenbank verwendet.

Weiterhin wird angenommen, dass das Datenvolumen gering ist und die Ausführungsdauer der Transformationen in einem akzeptablen Rahmen bleibt. Der Prozess ist nicht für produktive Workloads konzipiert, sondern dient als Proof of Concept und als Lernumgebung. Skalierbarkeit und Performance sind daher nachrangige Ziele.

Ein weiterer Aspekt ist die Verwendung von Open-Source-Werkzeugen und frei verfügbaren CI-Plattformen. Dadurch wird sichergestellt, dass der Ansatz ohne Lizenzkosten nachvollzogen und weiterentwickelt werden kann. Die Wahl von GitHub Actions als CI-Plattform erfolgt aufgrund der engen Integration mit Git-Repositories und der kostenlosen Verfügbarkeit für öffentliche Projekte.

### 3.2 Architekturübersicht

Die Architektur des CI-Prozesses gliedert sich in drei wesentliche Komponenten: das Git-Repository als Versionskontrolle, die CI-Plattform zur automatisierten Ausführung und die dateibasierte Datenbank als Ausführungsumgebung für Transformationen.

Das Git-Repository enthält sowohl die Quelldaten im CSV-Format als auch die dbt-Projektstruktur mit Modellen, Tests und Konfigurationsdateien. Bei jeder Änderung im Repository wird automatisch ein CI-Lauf ausgelöst. Dieser Lauf initialisiert eine virtuelle Umgebung, installiert die erforderlichen Abhängigkeiten und führt alle dbt-Transformationen sowie die definierten Tests aus.

Das Ergebnis des CI-Laufs wird protokolliert und kann direkt im Repository eingesehen werden. Schlägt ein Test fehl, wird dies unmittelbar sichtbar und kann analysiert werden. Auf diese Weise entsteht ein kontinuierlicher Feedback-Loop, der Qualitätsprobleme frühzeitig offenlegt.

### 3.3 Designentscheidungen

#### 3.3.1 Einsatz von DuckDB

DuckDB wurde als dateibasierte, spaltenorientierte Datenbank gewählt, da sie ohne Serverinstallation auskommt und vollständig in Prozessen läuft. Dies vereinfacht die Handhabung in einer CI-Umgebung erheblich. Zudem bietet DuckDB eine hohe Performance bei analytischen Abfragen und unterstützt SQL-Syntax, die mit vielen modernen Data Warehouses kompatibel ist.

Ein weiterer Vorteil von DuckDB ist die einfache Integration mit dbt. Es existiert ein offizieller dbt-Adapter, der die Ausführung von dbt-Projekten gegen DuckDB ermöglicht. Dies reduziert den Konfigurationsaufwand und erlaubt es, den Fokus auf die Modellierung und die Tests zu legen.

#### 3.3.2 Einsatz von dbt

dbt wurde als Transformationswerkzeug gewählt, weil es eine deklarative, modulare und testbare Herangehensweise an Datenmodellierung bietet. Die klare Strukturierung in Source, Staging und Mart-Schichten entspricht etablierten Best Practices im Data Engineering und erleichtert die Nachvollziehbarkeit.

Darüber hinaus bietet dbt native Unterstützung für Tests, Dokumentation und Versionierung. Diese Funktionen sind essenziell für einen CI-Prozess, da sie es ermöglichen, Qualitätskriterien explizit zu formulieren und automatisiert zu prüfen. Die Integration mit GitHub Actions ist gut dokumentiert und erleichtert die Umsetzung.

#### 3.3.3 Einsatz von GitHub Actions

GitHub Actions wurde als CI-Plattform gewählt, da es direkt in GitHub integriert ist und keine zusätzliche Infrastruktur erfordert. Die Definition von Workflows erfolgt deklarativ in YAML-Dateien, was Nachvollziehbarkeit und Wiederverwendbarkeit fördert. Zudem bietet GitHub Actions kostenlose Rechenzeit für öffentliche Repositories.

Ein weiterer Vorteil ist die Möglichkeit, Workflows bei verschiedenen Ereignissen auszulösen, etwa bei Pushes, Pull Requests oder zeitgesteuert. Dies erlaubt es, den CI-Prozess flexibel an unterschiedliche Szenarien anzupassen. Die Logs der Workflows sind transparent einsehbar und erleichtern die Fehleranalyse.

#### 3.3.4 Staging als View und Mart als Table

In der dbt-Projektstruktur wurde entschieden, Staging-Modelle als Views und Mart-Modelle als Tabellen zu materialisieren. Diese Entscheidung spiegelt eine bewusste Abwägung zwischen Performance und Flexibilität wider.

Staging-Modelle dienen der initialen Bereinigung und Typanpassung von Rohdaten. Da sie in der Regel einfache Transformationen sind und häufig als Zwischenschritt genutzt werden, ist eine Materialisierung als View ausreichend. Dies spart Speicherplatz und hält die Ausführungsdauer gering.

Mart-Modelle hingegen sind konsolidierte, oft aggregierte Datenmodelle, die für analytische Abfragen optimiert sind. Hier bietet sich eine Materialisierung als Tabelle an, um wiederholte Berechnungen zu vermeiden und schnellere Zugriffszeiten zu ermöglichen. In einem produktiven Setup würde diese Entscheidung je nach Nutzungsmuster variieren.

## 4. Technische Umsetzung

### 4.1 Projektstruktur

Das Projekt folgt einer standardisierten Verzeichnisstruktur, die sich an den Konventionen von dbt orientiert. Im Hauptverzeichnis befindet sich die dbt-Projektkonfiguration sowie die Quelldaten im CSV-Format. Der Ordner models enthält die Transformationslogik, unterteilt in die Schichten staging und mart.

Im Ordner staging werden die Rohdaten normalisiert und typisiert. Jede CSV-Datei wird durch ein Staging-Modell repräsentiert, das grundlegende Transformationen wie Spaltenumbenennungen und Datentyp-Konvertierungen durchführt. Diese Modelle sind als Views definiert und dienen als Basis für nachgelagerte Transformationen.

Im Ordner mart befinden sich konsolidierte Datenmodelle, die geschäftliche Fragestellungen abbilden. Diese Modelle aggregieren Daten aus mehreren Staging-Schichten und materialisieren diese als Tabellen. Zusätzlich sind im Ordner tests benutzerdefinierte Testfälle hinterlegt, die spezifische Datenqualitätskriterien prüfen.

### 4.2 Datenmodellierung mit dbt

Die Datenmodellierung erfolgt in SQL und nutzt die Jinja-Template-Syntax von dbt. Modelle referenzieren einander über die ref-Funktion, wodurch Abhängigkeiten explizit gemacht und von dbt automatisch aufgelöst werden. Jedes Modell enthält eine Beschreibung sowie Metadaten zu den verwendeten Spalten.

Ein Beispiel für ein Staging-Modell ist die Transformation einer Kundentabelle, bei der nur relevante Spalten ausgewählt, umbenannt und mit konsistenten Datentypen versehen werden. Ein nachgelagertes Mart-Modell könnte diese bereinigten Kundendaten mit Bestellungen verknüpfen und aggregierte Kennzahlen berechnen.

Durch die Verwendung von dbt-Konfigurationen kann für jedes Modell festgelegt werden, ob es als View oder als Tabelle materialisiert wird. Diese Konfiguration erfolgt entweder auf Modellebene oder global in der dbt-Projektkonfiguration. Die gewählte Strategie spiegelt die zuvor beschriebenen Designentscheidungen wider.

### 4.3 Definition von Datenqualitätstests

Zusätzlich zu den eingebauten Tests können benutzerdefinierte Tests als SQL-Abfragen formuliert werden. Ein solcher Test gibt bei Erfolg null Zeilen zurück und bei Fehlschlagen alle Zeilen, die die Bedingung verletzen. Dies ermöglicht eine hohe Flexibilität bei der Definition spezifischer Qualitätskriterien.

Im vorliegenden Projekt wurden unter anderem Tests definiert, die die Eindeutigkeit von Primärschlüsseln prüfen, die Vollständigkeit kritischer Spalten sicherstellen sowie referenzielle Integrität zwischen verknüpften Tabellen gewährleisten. Diese Tests tragen dazu bei, dass strukturelle Fehler frühzeitig erkannt werden.

### 4.4 CI Pipeline Ablauf

Die CI-Pipeline wird durch eine GitHub Actions Workflow-Datei definiert, die bei jedem Push auf den main-Branch ausgelöst wird. Der Workflow beginnt mit dem Checkout des Repositories, gefolgt von der Installation von Python und den erforderlichen Abhängigkeiten, insbesondere dbt und DuckDB.

Anschließend wird dbt deps ausgeführt, um eventuelle dbt-Pakete zu installieren. Danach folgt dbt seed, um die CSV-Quelldaten in die DuckDB-Datenbank zu laden. Nun werden die Transformationen mit dbt run durchgeführt, wodurch alle Modelle gemäß der definierten Abhängigkeiten ausgeführt werden.

Abschließend wird dbt test ausgeführt, um alle definierten Tests zu prüfen. Die Ergebnisse werden im Workflow-Log protokolliert. Schlägt ein Test fehl, bricht die Pipeline ab und markiert den Build als fehlgeschlagen. Dies gibt unmittelbares Feedback über die Datenqualität und die Funktionsfähigkeit der Transformationen.

### 4.5 Beispielhafter CI Lauf

Ein beispielhafter CI-Lauf beginnt mit dem Klonen des Repositories in die virtuelle Umgebung von GitHub Actions. Python wird in einer spezifizierten Version installiert, gefolgt von pip install dbt-core dbt-duckdb. Nach erfolgreicher Installation werden die dbt-Befehle sequenziell ausgeführt.

Die Ausgabe von dbt run zeigt an, welche Modelle erfolgreich erstellt wurden und wie viele Zeilen jeweils verarbeitet wurden. Bei dbt test werden alle definierten Tests durchlaufen und deren Status ausgegeben. Grüne Markierungen signalisieren erfolgreiche Tests, rote Markierungen weisen auf Fehler hin.

Im vorliegenden Projekt wurden sämtliche Tests erfolgreich durchlaufen, was die Korrektheit der Transformationslogik und die Einhaltung der definierten Qualitätskriterien bestätigt. Dieser erfolgreiche CI-Lauf dokumentiert den Zustand des Projekts zu einem bestimmten Zeitpunkt und dient als Referenz für zukünftige Änderungen.

## 5. Kritische Würdigung

### 5.1 Stärken des Ansatzes

Der vorgestellte Ansatz bietet mehrere wesentliche Vorteile. Er ist einfach nachvollziehbar, erfordert keine komplexe Infrastruktur und kann mit kostenfreien Werkzeugen umgesetzt werden. Die vollständige Versionierung aller Artefakte im Git-Repository ermöglicht Nachvollziehbarkeit und erleichtert die Zusammenarbeit in Teams.

Durch die automatisierte Ausführung von Tests bei jeder Änderung wird eine kontinuierliche Qualitätssicherung erreicht. Fehler werden frühzeitig erkannt und können behoben werden, bevor sie in nachgelagerte Systeme propagiert werden. Dies fördert eine Kultur der Qualität und Transparenz im Umgang mit Daten.

Darüber hinaus eignet sich der Ansatz hervorragend als Lernumgebung. Die verwendeten Werkzeuge sind weit verbreitet und gut dokumentiert. Studierende und Einsteigerinnen können anhand dieses Projekts grundlegende Prinzipien des Data Engineering sowie moderne DevOps-Praktiken erlernen. Die geringe Komplexität senkt die Einstiegshürde erheblich.

### 5.2 Limitationen

Trotz der Vorteile weist der Ansatz auch klare Limitationen auf. Die Verwendung einer dateibasierten Datenbank ist für produktive Workloads nicht geeignet, da Skalierbarkeit, Mehrbenutzerfähigkeit und Performance-Optimierung fehlen. Auch die Ausführungsdauer in einer CI-Umgebung ist bei größeren Datenmengen problematisch.

Ein weiteres Problem ist die fehlende Persistenz der Datenbank über CI-Läufe hinweg. Jedes Mal wird die Datenbank neu initialisiert, was inkrementelle Transformationen oder historische Analysen ausschließt. In einem produktiven Setup würde eine zentrale, persistente Datenbank verwendet, was jedoch zusätzliche Kosten und Komplexität mit sich bringt.

Zudem sind die definierten Tests begrenzt auf strukturelle und syntaktische Prüfungen. Inhaltliche oder semantische Datenqualitätsprobleme werden nicht zwangsläufig erkannt. Eine umfassende Data-Quality-Strategie würde zusätzliche Monitoring- und Alerting-Mechanismen erfordern, die über den Rahmen dieses Ansatzes hinausgehen.

### 5.3 Abgrenzung zu produktiven Setups

Ein produktives Setup würde eine cloud-basierte oder dedizierte Datenbank-Infrastruktur nutzen, etwa Snowflake, BigQuery oder ein gehostetes PostgreSQL-System. Dies ermöglicht bessere Performance, Skalierbarkeit und Mehrbenutzerzugriff. Zudem würden Orchestrierungs-Tools wie Airflow oder Dagster eingesetzt, um komplexe Abhängigkeiten und Zeitpläne zu verwalten.

Weiterhin würden in produktiven Umgebungen umfangreichere Monitoring- und Logging-Lösungen zum Einsatz kommen, etwa Datadog, Prometheus oder spezialisierte Data-Observability-Plattformen. Diese ermöglichen es, nicht nur strukturelle, sondern auch operative Probleme wie langsame Queries oder unerwartete Datenverteilungen zu erkennen.

Auch der CI-Prozess selbst würde in einer produktiven Umgebung erweitert. Neben automatisierten Tests würden Code-Reviews, manuelle Freigabeprozesse und Deployment-Pipelines implementiert. Zudem würden verschiedene Umgebungen (Entwicklung, Test, Produktion) klar getrennt, um Risiken zu minimieren.

Der vorliegende Ansatz ist somit bewusst als minimale Variante konzipiert, die die grundlegenden Prinzipien veranschaulicht, jedoch nicht alle Anforderungen eines produktiven Betriebs erfüllt. Er dient als Ausgangspunkt und Lernumgebung, nicht als fertige Lösung.

## 6. Fazit und Ausblick

Die vorliegende Arbeit hat gezeigt, wie ein minimaler CI-Prozess für dbt-Projekte konzipiert und umgesetzt werden kann. Durch den Einsatz von DuckDB, dbt und GitHub Actions wurde ein leichtgewichtiges Setup geschaffen, das die Prinzipien von Continuous Integration auf den Datenkontext überträgt. Die automatisierte Ausführung von Transformationen und Tests ermöglicht eine grundlegende Form von Observability und trägt zur Verbesserung der Datenqualität bei.

Die Ergebnisse zeigen, dass bereits mit einfachen Mitteln eine strukturierte Qualitätssicherung im Data Engineering erreicht werden kann. Der gewählte Ansatz ist transparent, nachvollziehbar und gut geeignet für Lernzwecke sowie für Prototyping-Phasen. Gleichzeitig wurden die Grenzen dieses Ansatzes deutlich: Skalierbarkeit, Performance und Produktionsreife sind nicht gegeben.

Für zukünftige Arbeiten bieten sich verschiedene Erweiterungen an. Eine Integration mit Cloud-basierten Data Warehouses würde eine realistischere Abbildung produktiver Szenarien ermöglichen. Auch die Einbindung von Orchestrierungs-Tools oder erweiterten Monitoring-Lösungen wäre denkbar. Zudem könnte untersucht werden, wie sich der Ansatz auf größere Datenmengen oder komplexere Transformationslogiken übertragen lässt.

Insgesamt leistet die Arbeit einen Beitrag zur Diskussion um Qualitätssicherung und DevOps-Praktiken im Data Engineering. Sie zeigt, dass moderne Werkzeuge und Methoden es ermöglichen, strukturierte und nachvollziehbare Datenprozesse zu schaffen, ohne dabei auf übermäßig komplexe Infrastruktur angewiesen zu sein. Dies ist insbesondere in frühen Projektphasen oder in ressourcenbeschränkten Kontexten von Wert.